In [7]:
pip install --upgrade gradio

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import gradio as gr
import pickle

# Load dataset
df = pd.read_csv("Downloads/CLOUDTHAT_FOLDER/Vehicle_Sales_2024_CSV.csv")

# Extract month
df['fromdate'] = pd.to_datetime(df['from_date'], errors='coerce')
df['month'] = df['fromdate'].dt.month

# Group by required columns
grouped = (
    df.groupby(['vehicle_class', 'model_desc', 'office_cd', 'month'])
      .size()
      .reset_index(name='registration_count')
)

# Encode categorical columns
encoders = {}
for col in ['vehicle_class', 'model_desc', 'office_cd']:
    le = LabelEncoder()
    grouped[col] = le.fit_transform(grouped[col])
    encoders[col] = le

# Define features and target
X = grouped[['vehicle_class', 'model_desc', 'office_cd', 'month']]
y = grouped['registration_count']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save model
with open("registration_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Save encoders
with open("label_encoders.pkl", "wb") as f:
    pickle.dump(encoders, f)

In [3]:
import gradio as gr
import pandas as pd
import pickle

# Load model and encoders
with open("registration_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("label_encoders.pkl", "rb") as f:
    encoders = pickle.load(f)

# Reverse lookup for dropdowns
vehicle_class_labels = encoders['vehicle_class'].classes_.tolist()
model_desc_labels = encoders['model_desc'].classes_.tolist()
office_cd_labels = encoders['office_cd'].classes_.tolist()
months = [str(i) for i in range(1, 13)]

# Prediction function
def predict(vehicle_class, model_desc, office_cd, month):
    try:
        input_df = pd.DataFrame([{
            'vehicle_class': encoders['vehicle_class'].transform([vehicle_class])[0],
            'model_desc': encoders['model_desc'].transform([model_desc])[0],
            'office_cd': encoders['office_cd'].transform([office_cd])[0],
            'month': int(month)
        }])
        prediction = model.predict(input_df)[0]
        return f"📈 Predicted Registrations: {int(prediction)}"
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio UI
iface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Dropdown(vehicle_class_labels, label="Vehicle Class"),
        gr.Dropdown(model_desc_labels, label="Model Description"),
        gr.Dropdown(office_cd_labels, label="Office Code"),
        gr.Dropdown(months, label="Month")
    ],
    outputs=gr.Textbox(label="Predicted Registration Count"),
    title="🚗 Vehicle Registration Predictor",
    description="""
Predicts the expected number of vehicle registrations based on:
- Vehicle class
- Model description
- Office code
- Month of registration

Model: Trained using RandomForestRegressor on grouped historical data.
"""
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [5]:
import gradio as gr
import pandas as pd
import pickle

# Load model and encoders
with open("registration_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("label_encoders.pkl", "rb") as f:
    encoders = pickle.load(f)

# Prepare dropdown choices with "All"
vehicle_class_choices = ["All"] + list(encoders['vehicle_class'].classes_)
model_desc_choices = ["All"] + list(encoders['model_desc'].classes_)
office_cd_choices = ["All"] + list(encoders['office_cd'].classes_)
month_choices = ["All"] + list(range(1, 13))

def predict(vehicle_class, model_desc, office_cd, month):
    results = []

    # If 'All' is selected for a feature, use all possible encoded values
    vehicle_class_list = encoders['vehicle_class'].classes_ if vehicle_class == "All" else [vehicle_class]
    model_desc_list = encoders['model_desc'].classes_ if model_desc == "All" else [model_desc]
    office_cd_list = encoders['office_cd'].classes_ if office_cd == "All" else [office_cd]
    month_list = list(range(1, 13)) if month == "All" else [month]

    # Create all combinations
    for vc in vehicle_class_list:
        for md in model_desc_list:
            for oc in office_cd_list:
                for m in month_list:
                    # Encode to numeric values
                    row = {
                        'vehicle_class': encoders['vehicle_class'].transform([vc])[0],
                        'model_desc': encoders['model_desc'].transform([md])[0],
                        'office_cd': encoders['office_cd'].transform([oc])[0],
                        'month': m
                    }
                    pred = model.predict(pd.DataFrame([row]))[0]
                    results.append({
                        'vehicle_class': vc,
                        'model_desc': md,
                        'office_cd': oc,
                        'month': m,
                        'prediction': round(pred, 2)
                    })

    # Return as DataFrame for nice table view
    return pd.DataFrame(results)

# Build the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## 🚗 Vehicle Registration Prediction")

    with gr.Row():
        vc_dd = gr.Dropdown(vehicle_class_choices, label="Vehicle Class", value=vehicle_class_choices[0])
        md_dd = gr.Dropdown(model_desc_choices, label="Model Description", value=model_desc_choices[0])
        oc_dd = gr.Dropdown(office_cd_choices, label="Office Code", value=office_cd_choices[0])
        m_dd = gr.Dropdown(month_choices, label="Month", value=month_choices[0])

    btn = gr.Button("Predict")
    output = gr.Dataframe(label="Predictions Table")

    btn.click(
        fn=predict,
        inputs=[vc_dd, md_dd, oc_dd, m_dd],
        outputs=[output]
    )

# Launch app
demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [31]:
demo.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* Running on public URL: https://c02693514107d40431.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
